In [34]:
def calculate_vars(da, biome, domain, land_area, cf_area, cf_time, units):
    
    mean_val = annual_mean(area_mean(da, biome, domain, cf_area, land_area), cf_time)

    average_vals = mean_val.mean(dim='year') 
    interannual_mean = mean_val.std(dim='year')

    data_var = da.name

    # save the reduced data
    out = xr.Dataset()
    out[f'{data_var}_mean'] = average_vals
    out[f'{data_var}_mean'].attrs= {'units': units,
                                 'long_name': da.attrs['long_name']}
    out[f'{data_var}_iav']  = interannual_mean
    out[f'{data_var}_iav'].attrs= {'units': units,
                                'long_name': da.attrs['long_name']}


    return out

In [36]:
def combine_history_vars(postp_dir, whit, vars):
    
    files = sorted([os.path.join(postp_dir, f) for f in os.listdir(postp_dir)])
    ds = get_ensemble(files, whit)
    ds = ds.chunk({'gridcell': 20, 'ens': 20, 'time': 20})
    
    data_vars = []
    for var in vars:
        var_ds = calculate_vars(ds[var], ds.biome, 'global', land_area,
                                cfs_area[var], cfs_time[var], var_units[var])
        var_ds['parameter'] = ds.parameter
        var_ds['minmax'] = ds.minmax
        data_vars.append(var_ds)
    ensemble_ds = xr.merge(data_vars)

    return ensemble_ds

In [42]:
clm_param_key_file = '/glade/work/afoster/FATES_calibration/parameter_files/clm6sp_oaat.csv' 
clm_param_key = pd.read_csv(clm_param_key_file, header=None)
clm_param_key.columns = ['key', 'param', 'minmax']

fates_param_key_file = '/glade/work/afoster/FATES_calibration/parameter_files/fates_param_oaat/fates_oaat_key.csv'
fates_param_key = pd.read_csv(fates_param_key_file, index_col=0)
fates_param_key.columns = ['key', 'minmax', 'param']

In [43]:
file_dict = {'clm_hydro': 
                {'top_dir': '/glade/derecho/scratch/afoster/FATES_calibration/ctsm_sp_oaat/archive',
                'postp_dir': '/glade/work/afoster/FATES_calibration/history_files/ctsm_sp_oaat',
                'out_file': '/glade/work/afoster/FATES_calibration/history_files/ctsm_sp_oaat.nc',
                'default_case': 'ctsm60SP_bigleaf_sparsegrid',
                'default_file': '/glade/work/afoster/FATES_calibration/history_files/ctsm_sp_default.nc',
                'fates_params': False,
                'fates': False},
            'clm_btran':
                {'top_dir': '/glade/derecho/scratch/afoster/FATES_calibration/ctsm_sp_oaat_btran/archive',
                'postp_dir': '/glade/work/afoster/FATES_calibration/history_files/ctsm_sp_oaat_btran',
                'out_file': '/glade/work/afoster/FATES_calibration/history_files/ctsm_sp_oaat_btran.nc',
                'default_case': 'ctsm60SP_bigleaf_sparsegrid_btran',
                'default_file': '/glade/work/afoster/FATES_calibration/history_files/ctsm_sp_default_btran.nc',
                'fates_params': False,
                'fates': False},
            'fates_oaat':
                {'top_dir': '/glade/derecho/scratch/afoster/FATES_calibration/fates_sp_oaat/archive',
                'postp_dir': '/glade/work/afoster/FATES_calibration/history_files/fates_sp_oaat',
                'out_file': '/glade/work/afoster/FATES_calibration/history_files/fates_sp_oaat.nc',
                'default_case': 'ctsm60SP_fates_sparsegrid',
                'default_file': '/glade/work/afoster/FATES_calibration/history_files/fates_sp_oaat_default.nc',
                'fates_params': True,
                'fates': True},
            'fates_clmpars_oaat':
                {'top_dir': '/glade/derecho/scratch/afoster/FATES_calibration/fates_clmpars_sp_oaat/archive',
                'postp_dir': '/glade/work/afoster/FATES_calibration/history_files/fates_clmpars_sp_oaat',
                'out_file': '/glade/work/afoster/FATES_calibration/history_files/fates_clmpars_sp_oaat.nc',
                'default_case': None,
                'default_file': None,
                'fates_params': False,
                'fates': True},
            }

In [49]:
simulation = 'fates_clmpars_oaat'

In [53]:
if file_dict[simulation]['fates']:
    data_vars = fates_data_vars
else:
    data_vars = clm_data_vars
if file_dict[simulation]['fates_params']:
    key = fates_param_key
else:
    key = clm_param_key

In [51]:
if file_dict[simulation]['default_case'] is not None:
    default_ds = get_default('/glade/derecho/scratch/afoster/archive',
                             file_dict[simulation]['default_case'],
                             data_vars, key, whit, out_vars,
                             fates=file_dict[simulation]['fates'])
    default_ds.to_netcdf(file_dict[simulation]['default_file'])

In [ ]:
not_finished = write_history_files(file_dict[simulation]['top_dir'], data_vars,
                                   key, file_dict[simulation]['postp_dir'],
                                   fates=file_dict[simulation]['fates'],
                                   fates_params=file_dict[simulation]['fates_params'])

In [47]:
ensemble_ds = combine_history_vars(file_dict[simulation]['postp_dir'], whit, out_vars)

In [48]:
ensemble_ds.to_netcdf(file_dict[simulation]['out_file'])

/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 180.01 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
client.shutdown()